In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入mnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


----------------------------
Train_z, Test_z
PCA: 0.9507
SVD: 0.9486
NetD, Acc at first 0.9488


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.53s/it]


----------------------------
Train_z, Test_z
PCA: 0.935
SVD: 0.9317
NetG, Acc at first 0.9572


In [5]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [ ]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 5

for epoch in range(num_steps):
    if 1==1:
        
        #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        #radius_list.append(radius)
        #print(radius)

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            #radius_list.append(radius)
            #print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        

tensor(-43.1898, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-3.8196, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3242, device='cuda:0', grad_fn=<NegBackward0>), tensor(-35.0460, device='cuda:0', grad_fn=<AddBackward0>)]
7.846552576711052
tensor(-43.8656, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.5653, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.9531, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.3472, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-39.6048, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.5653, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.0652, device='cuda:0', grad_fn=<NegBackward0>), tensor(-32.9743, device='cuda:0', grad_fn=<AddBackward0>)]
99.96949314545007
tensor(-63.3742, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-7.0303, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.6227, device='cuda:0', grad_fn=<NegBackward0>), tensor(-51.7212, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-65.6700, device='cud

tensor(-97.6320, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.3096, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.6507, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.6717, device='cuda:0', grad_fn=<AddBackward0>)]
5.070750083359844
tensor(-100.2958, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.5790, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4774, device='cuda:0', grad_fn=<NegBackward0>), tensor(-77.2394, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-95.5317, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.5790, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.6035, device='cuda:0', grad_fn=<NegBackward0>), tensor(-74.3491, device='cuda:0', grad_fn=<AddBackward0>)]
5.126337837069265
tensor(-97.6304, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-14.9427, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.2030, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.4846, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-98.4907, device

5.327527079544279
tensor(-105.2171, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.9101, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.8993, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.4077, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-106.8082, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-18.9101, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4900, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.4081, device='cuda:0', grad_fn=<AddBackward0>)]
5.367860870134082
tensor(-105.4844, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.0843, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7513, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.6489, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-106.8388, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.0843, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3253, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.4292, device='cuda:0', grad_fn=<AddBackward0>)]
4.506224305887209
te

5.176760636420036
tensor(-108.6974, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.1132, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.2942, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2900, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-107.1462, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.1132, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.9855, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.0475, device='cuda:0', grad_fn=<AddBackward0>)]
5.465075179659453
tensor(-106.6528, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.4617, device='cuda:0', grad_fn=<NegBackward0>), tensor(-6.0491, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.1420, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-109.0485, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.4617, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.2304, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.3564, device='cuda:0', grad_fn=<AddBackward0>)]
4.673996122988731
te

4.797232529276472
tensor(-112.2812, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0117, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.9875, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2820, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-111.5677, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0117, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.1427, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.4133, device='cuda:0', grad_fn=<AddBackward0>)]
4.899588035626932
tensor(-116.3284, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.1067, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.3883, device='cuda:0', grad_fn=<NegBackward0>), tensor(-81.8333, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-114.8525, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.1067, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3975, device='cuda:0', grad_fn=<NegBackward0>), tensor(-81.3483, device='cuda:0', grad_fn=<AddBackward0>)]
4.956676492381282
te

...

4.145558347530671
tensor(-136.9344, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2455, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.3499, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.3390, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-136.2080, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2455, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.4813, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.4813, device='cuda:0', grad_fn=<AddBackward0>)]
5.191161543557647
tensor(-134.3461, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1842, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3576, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.8043, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-136.8851, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1842, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.7858, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.9151, device='cuda:0', grad_fn=<AddBackward0>)]
4.553195045195836
te

4.905386920897746
tensor(-135.9696, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1158, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.9385, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.9153, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-136.7555, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1158, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.2475, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.3923, device='cuda:0', grad_fn=<AddBackward0>)]
4.835918968073833
tensor(-135.5879, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1200, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.0717, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.3961, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-139.2229, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1200, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.7828, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.3200, device='cuda:0', grad_fn=<AddBackward0>)]
4.64814645104654
ten

5.644999066072624
tensor(-139.6172, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1007, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.4254, device='cuda:0', grad_fn=<NegBackward0>), tensor(-92.0912, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-135.4495, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1007, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.9218, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.4271, device='cuda:0', grad_fn=<AddBackward0>)]
4.219093531374525
tensor(-138.5962, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2589, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.6125, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.7248, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-136.0038, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2589, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.4757, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.2692, device='cuda:0', grad_fn=<AddBackward0>)]
4.237367290335155
t

4.313897117889177
tensor(-132.8077, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2960, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3667, device='cuda:0', grad_fn=<NegBackward0>), tensor(-86.1450, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-137.4878, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2960, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.4107, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.7810, device='cuda:0', grad_fn=<AddBackward0>)]
4.931400019645829
tensor(-135.6022, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1288, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.4076, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.0658, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-134.6129, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1288, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.3476, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.1366, device='cuda:0', grad_fn=<AddBackward0>)]
4.975944195813191
t

4.199236111966814
tensor(-134.7102, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3160, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.0685, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.3257, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-137.3134, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3160, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.7778, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.2196, device='cuda:0', grad_fn=<AddBackward0>)]
4.504020855450686
tensor(-134.5666, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1189, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.9377, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.5100, device='cuda:0', grad_fn=<AddBackward0>)]
tensor(-138.3696, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1189, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.5153, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.7354, device='cuda:0', grad_fn=<AddBackward0>)]
4.594456538988715


In [ ]:
radius_list

In [7]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.52s/it]


----------------------------
Train_z, Test_z
PCA: 0.9416
SVD: 0.9272
NetD, Acc at first 0.9643


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.83s/it]


----------------------------
Train_z, Test_z
PCA: 0.9262
SVD: 0.9238
NetG, Acc at first 0.9542


In [ ]:
# a broke happens during training hence not repeating training, wait till all finished.
torch.save(netD, 'MNIST_NetD-mode1-radius1.pth')
torch.save(netV, 'MNIST_NetV-mode1-radius1.pth')
torch.save(netG, 'MNIST_NetG-mode1-radius1.pth')

### 0-2
netD = torch.load('MNIST_NetD-mode1-radius1.pth')
netV = torch.load('MNIST_NetV-mode1-radius1.pth')
netG = torch.load('MNIST_NetG-mode1-radius1.pth')

mode = 1
gam3 = 1

In [8]:
#robust testing
r_list = [0,0.1,0.2,0.5,1,2,3,4,6,8,10]
sample_num = 20

In [9]:
# sample 20 times of NetG at final radius, NetD is fixed, length 0.1,0.2,0.5,1,2,4,6,8,10,15,20
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample = []
for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[1])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample.append(count/10000)

print(max(Acc_results_G_sample),min(Acc_results_G_sample),np.mean(Acc_results_G_sample),np.var(Acc_results_G_sample))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9339
SVD: 0.9194
NetG_fixD, Acc at sample 1 0.9692


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9332
SVD: 0.9189
NetG_fixD, Acc at sample 2 0.9698


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9339
SVD: 0.9197
NetG_fixD, Acc at sample 18 0.9696


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9343
SVD: 0.9194
NetG_fixD, Acc at sample 19 0.9696


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.78s/it]


----------------------------
Train_z, Test_z
PCA: 0.9331
SVD: 0.9194
NetG_fixD, Acc at sample 20 0.9697
0.9699 0.9688 0.969495 8.447500000000227e-08


In [10]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample2 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[2])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample2.append(count/10000)
    
print(max(Acc_results_G_sample2),min(Acc_results_G_sample2),np.mean(Acc_results_G_sample2),np.var(Acc_results_G_sample2))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


----------------------------
Train_z, Test_z
PCA: 0.9342
SVD: 0.9186
NetG_fixD, Acc at sample 1 0.9696


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9329
SVD: 0.9171
NetG_fixD, Acc at sample 2 0.9696


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9337
SVD: 0.9197
NetG_fixD, Acc at sample 18 0.9706


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.82s/it]


----------------------------
Train_z, Test_z
PCA: 0.9341
SVD: 0.9192
NetG_fixD, Acc at sample 19 0.9695


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


----------------------------
Train_z, Test_z
PCA: 0.9351
SVD: 0.9204
NetG_fixD, Acc at sample 20 0.9695
0.9706 0.9687 0.969565 2.1727499999999758e-07


In [11]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample3 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[3])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample3.append(count/10000)
    
print(max(Acc_results_G_sample3),min(Acc_results_G_sample3),np.mean(Acc_results_G_sample3),np.var(Acc_results_G_sample3))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.79s/it]


----------------------------
Train_z, Test_z
PCA: 0.933
SVD: 0.9177
NetG_fixD, Acc at sample 1 0.9682


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9337
SVD: 0.9222
NetG_fixD, Acc at sample 2 0.9686


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9319
SVD: 0.9212
NetG_fixD, Acc at sample 18 0.9692


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.933
SVD: 0.9197
NetG_fixD, Acc at sample 19 0.9698


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9312
SVD: 0.9212
NetG_fixD, Acc at sample 20 0.9688
0.9699 0.9681 0.9688900000000003 2.6190000000000014e-07


In [12]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample4 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[4])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample4.append(count/10000)
    
print(max(Acc_results_G_sample4),min(Acc_results_G_sample4),np.mean(Acc_results_G_sample4),np.var(Acc_results_G_sample4))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9324
SVD: 0.9238
NetG_fixD, Acc at sample 1 0.9662


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.66s/it]


----------------------------
Train_z, Test_z
PCA: 0.9279
SVD: 0.9197
NetG_fixD, Acc at sample 2 0.9669


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9339999999999999
SVD: 0.924
NetG_fixD, Acc at sample 18 0.9672


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.76s/it]


----------------------------
Train_z, Test_z
PCA: 0.9311
SVD: 0.924
NetG_fixD, Acc at sample 19 0.9661


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9336
SVD: 0.923
NetG_fixD, Acc at sample 20 0.967
0.968 0.9655 0.9669549999999999 4.414750000000019e-07


In [13]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample5 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[5])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample5.append(count/10000)
    
print(max(Acc_results_G_sample5),min(Acc_results_G_sample5),np.mean(Acc_results_G_sample5),np.var(Acc_results_G_sample5))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.9256
SVD: 0.9171
NetG_fixD, Acc at sample 1 0.9604


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


----------------------------
Train_z, Test_z
PCA: 0.9286
SVD: 0.9224
NetG_fixD, Acc at sample 2 0.9579


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9325
SVD: 0.9246
NetG_fixD, Acc at sample 18 0.9611


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.74s/it]


----------------------------
Train_z, Test_z
PCA: 0.9286
SVD: 0.9237
NetG_fixD, Acc at sample 19 0.9601


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


----------------------------
Train_z, Test_z
PCA: 0.938
SVD: 0.9306
NetG_fixD, Acc at sample 20 0.958
0.9619 0.9549 0.9587649999999999 4.2462749999999876e-06


In [14]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample6 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[6])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample6.append(count/10000)
    
print(max(Acc_results_G_sample6),min(Acc_results_G_sample6),np.mean(Acc_results_G_sample6),np.var(Acc_results_G_sample6))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.74s/it]


----------------------------
Train_z, Test_z
PCA: 0.9277
SVD: 0.9224
NetG_fixD, Acc at sample 1 0.8463


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.68s/it]


----------------------------
Train_z, Test_z
PCA: 0.9262
SVD: 0.9207
NetG_fixD, Acc at sample 2 0.8413


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9365
SVD: 0.9332
NetG_fixD, Acc at sample 18 0.9028


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


----------------------------
Train_z, Test_z
PCA: 0.9325
SVD: 0.9299999999999999
NetG_fixD, Acc at sample 19 0.8919


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.76s/it]


----------------------------
Train_z, Test_z
PCA: 0.9285
SVD: 0.9252
NetG_fixD, Acc at sample 20 0.8697
0.9287 0.775 0.8715150000000002 0.0012599582749999994


In [15]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample7 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[7])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample7.append(count/10000)
    
print(max(Acc_results_G_sample7),min(Acc_results_G_sample7),np.mean(Acc_results_G_sample7),np.var(Acc_results_G_sample7))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.74s/it]


----------------------------
Train_z, Test_z
PCA: 0.9289000000000001
SVD: 0.9233
NetG_fixD, Acc at sample 1 0.3979


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.81s/it]


----------------------------
Train_z, Test_z
PCA: 0.9263
SVD: 0.923
NetG_fixD, Acc at sample 2 0.4841


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9231
SVD: 0.918
NetG_fixD, Acc at sample 18 0.4036


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


----------------------------
Train_z, Test_z
PCA: 0.9369
SVD: 0.9311
NetG_fixD, Acc at sample 19 0.483


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.71s/it]


----------------------------
Train_z, Test_z
PCA: 0.9328
SVD: 0.9286
NetG_fixD, Acc at sample 20 0.6143
0.6143 0.2762 0.475785 0.0055542122749999995


In [16]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample8 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[8])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample8.append(count/10000)
    
print(max(Acc_results_G_sample8),min(Acc_results_G_sample8),np.mean(Acc_results_G_sample8),np.var(Acc_results_G_sample8))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.69s/it]


----------------------------
Train_z, Test_z
PCA: 0.9287
SVD: 0.9261
NetG_fixD, Acc at sample 1 0.1341


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.70s/it]


----------------------------
Train_z, Test_z
PCA: 0.9267
SVD: 0.9238999999999999
NetG_fixD, Acc at sample 2 0.1114


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.73s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9254
SVD: 0.9216
NetG_fixD, Acc at sample 18 0.1634


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.77s/it]


----------------------------
Train_z, Test_z
PCA: 0.9297
SVD: 0.9286
NetG_fixD, Acc at sample 19 0.1491


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.88s/it]


----------------------------
Train_z, Test_z
PCA: 0.923
SVD: 0.9213
NetG_fixD, Acc at sample 20 0.144
0.1828 0.1101 0.14353500000000002 0.000269334275


In [17]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample9 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[9])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample9.append(count/10000)
    
print(max(Acc_results_G_sample9),min(Acc_results_G_sample9),np.mean(Acc_results_G_sample9),np.var(Acc_results_G_sample9))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.90s/it]


----------------------------
Train_z, Test_z
PCA: 0.9288
SVD: 0.9245
NetG_fixD, Acc at sample 1 0.1122


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.85s/it]


----------------------------
Train_z, Test_z
PCA: 0.9338
SVD: 0.9315
NetG_fixD, Acc at sample 2 0.0977


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.86s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9271
SVD: 0.9211
NetG_fixD, Acc at sample 18 0.1014


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.84s/it]


----------------------------
Train_z, Test_z
PCA: 0.9336
SVD: 0.9303
NetG_fixD, Acc at sample 19 0.1078


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.91s/it]


----------------------------
Train_z, Test_z
PCA: 0.9256
SVD: 0.919
NetG_fixD, Acc at sample 20 0.1357
0.154 0.0777 0.11134 0.0003214063999999999


In [18]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample10 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[10])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample10.append(count/10000)
    
print(max(Acc_results_G_sample10),min(Acc_results_G_sample10),np.mean(Acc_results_G_sample10),np.var(Acc_results_G_sample10))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.92s/it]


----------------------------
Train_z, Test_z
PCA: 0.9322
SVD: 0.9291
NetG_fixD, Acc at sample 1 0.1266


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.87s/it]


----------------------------
Train_z, Test_z
PCA: 0.9289000000000001
SVD: 0.9227
NetG_fixD, Acc at sample 2 0.0973


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.86s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9264
SVD: 0.9243
NetG_fixD, Acc at sample 18 0.0791


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.77s/it]


----------------------------
Train_z, Test_z
PCA: 0.9221
SVD: 0.9198
NetG_fixD, Acc at sample 19 0.0954


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.81s/it]


----------------------------
Train_z, Test_z
PCA: 0.9236
SVD: 0.9188000000000001
NetG_fixD, Acc at sample 20 0.0982
0.1354 0.0714 0.10324499999999999 0.000304375475
